# Importing libraries

In [1]:
#Common Lib.
import os
import cv2
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import warnings
warnings.filterwarnings('ignore')


#Deep Learning Lib / Image to text
from tensorflow.keras.models import load_model
from imutils.contours import sort_contours
import imutils
from PIL import Image, ImageEnhance, ImageFilter
from IPython.display import clear_output
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"  # Location of tesseract folder


#Text to Speech Lib

from gtts import gTTS 
import googletrans
from googletrans import Translator, constants
from playsound import playsound 

## Image type selection 

In [2]:
def image_selection():
    selection = input("Select image type : \n 1.Handwritten Text \n 2.Document\n ")
    return int(selection)

## Image capture through WEBCAM

In [3]:
def capture_image():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("test")

    img_counter = 0

    while True :
        ret, frame = cam.read()
        if not ret:
            print("failed to grab frame")
            break
        cv2.imshow("test", frame)

        k = cv2.waitKey(1)
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
           # img_name = ("captured-"+ date_string + ".jpg")
            date_string = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
#             print(date_string)
            img_dir = "./captured_images/captured{}.jpg".format(date_string)
            cv2.imwrite(img_dir , frame)
           # print("{} written!")
            img_counter += 1
            if (img_counter == 1):
                break

    cam.release()
    cv2.destroyAllWindows()
    
    return img_dir

## Image conversion to text

In [4]:
def image_conversion(img_dir):

    img = cv2.imread(img_dir)
    text = pytesseract.image_to_string(img)
    
    return text

## Language Codes Dataframe

In [5]:
dict_lan = googletrans.LANGCODES
df = pd.DataFrame.from_dict(dict_lan, orient='index' , columns=["Language Code" ] )
code=np.arange(1,107,1)
df["Code"]=code
#df.to_csv("./Languages_code.csv")

In [6]:
def language_codes():
    os.startfile("Languages_code.csv")

## Language Selector

In [7]:
def lan_selector(code):
    temp_list = list(dict_lan.values())
    if code == 0:
        lan = temp_list[0]
    else:
        lan = temp_list[code-1]
    return lan

## Text Translation

In [8]:
def text_translator(text ,lan ):
    translator = Translator()
    translated = translator.translate(text ,dest = lan)
    translated_text = translated.text

    return translated_text

In [9]:
def display_text(text):
    print(text)

## Saving And Playing audio

In [10]:
# Check language
def save_tran_text(translated_text,lan):
    date_string = datetime.datetime.now().strftime("%Y-%m-%d-%H-%M")
    saved = gTTS(text = translated_text,lang =lan) 
    sound_dir = "./saved_sounds/Translated-{}.mp3".format(date_string)
    saved.save(sound_dir)
    return sound_dir

In [11]:
def sound_playback(sound_dir):
    playsound(sound_dir)

## Handwritten Text Detection

In [12]:
def handwritten_text(image):
    handwritten_model = load_model("model.h5")
    image = cv2.imread(image)
#     # IMAGES for clarification
#     cv2.imshow("Orignal Image ", image)
#     cv2.waitKey(0)
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     cv2.imshow("image after grayscale", gray)
#     cv2.waitKey(0)
#     blurred = cv2.GaussianBlur(gray, (5, 5), 0)
#     cv2.imshow("image after blurring", blurred)
#     cv2.waitKey(0)
#     edged = cv2.Canny(blurred, 30, 150)
#     cv2.imshow("image after edge-detection", edged)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()
    
    
    
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edged = cv2.Canny(blurred, 30, 150)
    cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    cnts = sort_contours(cnts, method="left-to-right")[0]
    
    chars = []
    chars_alt = []
    for c in cnts:
        (x, y, w, h) = cv2.boundingRect(c)      # compute the bounding box of the contour
        if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):
            roi = gray[y:y + h, x:x + w]
            thresh = cv2.threshold(roi, 0, 255,cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
            (tH, tW) = thresh.shape
            
            if tW > tH:
                thresh = imutils.resize(thresh, width=32)
            else :
                thresh = imutils.resize(thresh, width=32)
                
            (tH, tW) = thresh.shape  #Updated shape
            dX = int(max(0, 32 - tW) / 2.0)
            dY = int(max(0, 32 - tH) / 2.0)
            padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,left=dX, right=dX,
                                        borderType=cv2.BORDER_CONSTANT, value=(0, 0, 0))
            padded = cv2.resize(padded, (32, 32))
            padded = padded.astype("float32") / 255.0
            padded = np.expand_dims(padded, axis=-1)
            chars_alt.append((padded, (x, y, w, h)))
            chars= np.array(chars_alt)
            #chars = np.array(chara)
            
            boxes = [b[1] for b in chars]
            chars = np.array([c[0] for c in chars], dtype="float32")
            preds = handwritten_model.predict(chars)
            
            
            
            labelNames = "0123456789"
            labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
            labelNames = [l for l in labelNames]
            output_string = ""
            probi=[]
            for (pred, (x, y, w, h)) in zip(preds, boxes):
                i = np.argmax(pred)
                prob = pred[i]
                label = labelNames[i]
                output_string += labelNames[i]
                probi.append(pred[i]*100)                
                #print("[INFO] {} - {:.2f}%".format(label, prob * 100))
                cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(image, label, (x - 10, y - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)
                #cv2.imshow("Image with contours", image)
                #cv2.waitKey(0)
            cv2.imshow("Final Image",image)
            cv2.waitKey(0)
            
#     cv2.imshow("Final Image_",image)
#     cv2.waitKey(0)
    cv2.destroyAllWindows()
    print("The accuracies are :")
    for i in range(len(probi)):
        print("{} = {:.2f}%".format(output_string[i],float(probi[i])))
    return output_string        

# Main Function 

In [13]:
selection = image_selection()
if selection == 1:   
    clear_output()        # CLEAR SCREEN
    
    img_type = input("Image selection \n For webcam selection press 1 \n For file Directory press 2\n" ) 
    img_type = int(img_type)
    
    clear_output()        # CLEAR SCREEN
    
    if img_type == 1:
        img_dir = capture_image()
        output = handwritten_text(img_dir)
        print("The text detected is {}".format(output))
        
    elif img_type==2:
        img_dir= input("Enter Image directory")
        clear_output()        # CLEAR SCREEN
        output = handwritten_text(img_dir)
        print("The text detected is {}".format(output)) 
elif selection == 2:    
    clear_output()        # CLEAR SCREEN      
    img_type = input("Image selection \n For webcam selection press 1 \n For file Directory press 2\n" ) 
    img_type = int(img_type)
    
    clear_output()        # CLEAR SCREEN
    
    if img_type == 1:
        img_dir = capture_image()
        print(img_dir)
        
        clear_output()        # CLEAR SCREEN
        
        text = image_conversion(img_dir)
        print("Text Detected")
        display_text(text)
        language_codes()
        print("Check for language code in the csv file just loaded!!!!")
        code = input("Enter language code for translation\n")
        
        clear_output()        # CLEAR SCREEN
        
        lan = lan_selector(int(code))
        translated_text = text_translator(text,lan)
        print(translated_text)    
        print("Do you want to hear the translated text")
        
        clear_output()        # CLEAR SCREEN
        
        i = input("Press 1 for yes \nPress 2 for no\n")
        i = int(i)
        
        clear_output()        # CLEAR SCREEN
        
        if i ==1:
            print("Please Wait......Translating")
            sound_dir = save_tran_text(translated_text,lan)
            
            clear_output()        # CLEAR SCREEN
            
            sound_dir_loc=sound_dir[1:]
            print("Your translated sound is also saved at C:/Users/Desktop/project{}\nEND".format(sound_dir_loc))
            
            playsound(sound_dir)
        else:
            print("Thanks.....END!!!")
        
    elif img_type == 2 :
        img_dir = input("Enter Image path\n")
        text = image_conversion(img_dir)
        
        clear_output()        # CLEAR SCREEN
        
        print("Text Detected")
        display_text(text)
        language_codes()
        print("Check for language code in the csv file just loaded!!!!")
        code = input("Enter language code for translation\n")
        
        clear_output()        # CLEAR SCREEN
        
        lan = lan_selector(int(code))
        translated_text = text_translator(text,lan)
        print(translated_text)
        print("Do you want to hear the translated text")
        i = input("Press 1 for yes \nPress 2 for no\n")
        i = int(i)
        
        clear_output()        # CLEAR SCREEN
        
        if i ==1:
            print("Please Wait......Translating")
            sound_dir = save_tran_text(translated_text,lan)
            
            clear_output()        # CLEAR SCREEN
            
            sound_dir_loc=sound_dir[1:]
            print("Your translated sound is avl at C:/Users/Desktop/project{}\nEND".format(sound_dir_loc))
            playsound(sound_dir)
        elif i ==2 :
            print("Thanks... END!!!!!!!!") 

Your translated sound is avl at C:/Users/Desktop/project/saved_sounds/Translated-2021-02-21-12-30.mp3
END
